In [1]:
import pandas as pd
import numpy as np 
import os
import sys
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
import statistics 
from statistics import mode 

In [2]:
btcusd = pd.read_csv('../Binance/crypto_pumps/data/raw/Binance/BTC_USDT.csv')

In [3]:
btcusd.head()

,1518170354789,7789.9,8230.46,7789.9.1,8230.46.1,148.475418
0,1518170414789,8230.46,8317.74,8230.45,8317.74,91.494282
1,1518170474789,8304.00,8369.00,8304.00,8350.00,131.872916
2,1518170534789,8346.50,8380.00,8330.00,8380.00,84.544616
3,1518170594789,8379.00,8390.00,8348.00,8360.02,112.940304
4,1518170654789,8350.04,8389.00,8348.00,8348.13,88.138218


In [4]:
btcusd.columns = ['timestamp','open','high','low','close','volume']
btcusd.head()

,timestamp,open,high,low,close,volume
0,1518170414789,8230.46,8317.74,8230.45,8317.74,91.494282
1,1518170474789,8304.00,8369.00,8304.00,8350.00,131.872916
2,1518170534789,8346.50,8380.00,8330.00,8380.00,84.544616
3,1518170594789,8379.00,8390.00,8348.00,8360.02,112.940304
4,1518170654789,8350.04,8389.00,8348.00,8348.13,88.138218


In [5]:
btcusd.columns = ['timestamp','open','high','low','close','volume']
btcusd['date'] = pd.to_datetime(btcusd['timestamp'], unit = 'ms')
btcusd = btcusd.set_index(pd.DatetimeIndex(btcusd['date']))
btcusd = btcusd.drop(['date'],axis = 1)
btcusd.head()

btcusd['returns'] = btcusd['open'].pct_change()

In [6]:
btcusd['returns'] = btcusd['open'].pct_change()
btcusd = btcusd.iloc[1:,:]
#btcusd = btcusd.reset_index(drop = True)
btcusd.head()

,timestamp,open,high,low,close,volume,returns
date,,,,,,,
2018-02-09 10:01:14.789,1518170474789,8304.00,8369.00,8304.0,8350.00,131.872916,0.008935
2018-02-09 10:02:14.789,1518170534789,8346.50,8380.00,8330.0,8380.00,84.544616,0.005118
2018-02-09 10:03:14.789,1518170594789,8379.00,8390.00,8348.0,8360.02,112.940304,0.003894
2018-02-09 10:04:14.789,1518170654789,8350.04,8389.00,8348.0,8348.13,88.138218,-0.003456
2018-02-09 10:05:14.789,1518170714789,8350.00,8369.99,8300.0,8338.43,74.700930,-0.000005


In [7]:
btcusd[['returns']].describe()

,returns
count,1.572062e+06
mean,1.591032e-06
std,1.090750e-03
min,-7.117457e-02
25%,-3.628313e-04
50%,0.000000e+00
75%,3.654305e-04
max,7.765685e-02


In [8]:
def load_resample_df(path,time_measure):
    df = pd.read_csv(path)
    df.columns = ['timestamp','open','high','low','close','volume']
    df['date'] = pd.to_datetime(df['timestamp'], unit = 'ms')
    df = df.set_index(pd.DatetimeIndex(btcusd['date']))
    df = df.drop(['date'],axis = 1)        
     
    _,_ = check_for_missing_timestamps(df,interval_length)
    df = resample(df,time_measure)
    
    df['returns'] = df['open']
    
    df['returns'] = df['open'].pct_change()
    
    return(df)

## Check for missing values 

This is not only for missing entries within the given data but also 
periods of time for which no records are to be found in the downloaded dataset - either 
the exchange was down or the records are unavailable for download. 

In [9]:
btcusd.isnull().sum().sum()

0

In [10]:
X = btcusd['timestamp'].values
l_temp = [X[i]-X[i-1] for i in range(1,len(X))]
mode(l_temp)

60000

Only one missing value - the first entry of the returns column. 

In [16]:
def check_for_missing_timestamps(df):
    m = len(df)
    X = df['timestamp'].values
    count = 0
    missing_indices = []
    
    l_temp = [X[i]-X[i-1] for i in range(1,len(X))]
    interval_length = mode(l_temp)    # We assume that we more or less have all the data. 
    
    for i in range(1,m):
        if X[i]-X[i-1] != interval_length:
            count += 1
            error = np.timedelta64(X[i]-X[i-1],'ms')
            missing_indices.append((i,error))
    
    t = np.array([x[1] for x in missing_indices]).sum()
    t = t.astype('int')
    t = t/60000 
    
    print('Number of instances for which we have intervals of missing data is {}.'.format(count))
    print('')
    print('Total amount of missing time in dataframe is {} minutes'.format(t))
    
    return(count,missing_indices)        

In [17]:
check_for_missing_timestamps(btcusd)

Number of instances for which we have intervals of missing data is 22.

Total amount of missing time in dataframe is 4878.753516666667 minutes


(22,
 [(1199, numpy.timedelta64(945211,'ms')),
  (2505, numpy.timedelta64(2100000,'ms')),
  (196750, numpy.timedelta64(36060000,'ms')),
  (198250, numpy.timedelta64(6360000,'ms')),
  (207468, numpy.timedelta64(27480000,'ms')),
  (361428, numpy.timedelta64(12660000,'ms')),
  (398418, numpy.timedelta64(25260000,'ms')),
  (567918, numpy.timedelta64(21660000,'ms')),
  (659778, numpy.timedelta64(36060000,'ms')),
  (693392, numpy.timedelta64(3720000,'ms')),
  (791537, numpy.timedelta64(28860000,'ms')),
  (920657, numpy.timedelta64(8460000,'ms')),
  (920727, numpy.timedelta64(240000,'ms')),
  (937794, numpy.timedelta64(7260000,'ms')),
  (1047114, numpy.timedelta64(3660000,'ms')),
  (1062030, numpy.timedelta64(21300000,'ms')),
  (1081702, numpy.timedelta64(7740000,'ms')),
  (1156012, numpy.timedelta64(9060000,'ms')),
  (1248022, numpy.timedelta64(12660000,'ms')),
  (1471252, numpy.timedelta64(3660000,'ms')),
  (1501922, numpy.timedelta64(13860000,'ms')),
  (1506722, numpy.timedelta64(3660000,'

In [19]:
count,missing_indices = check_for_missing_timestamps(btcusd)

Number of instances for which we have intervals of missing data is 22.

Total amount of missing time in dataframe is 4878.753516666667 minutes


In [20]:
print(count)
missing_indices

22


[(1199, numpy.timedelta64(945211,'ms')),
 (2505, numpy.timedelta64(2100000,'ms')),
 (196750, numpy.timedelta64(36060000,'ms')),
 (198250, numpy.timedelta64(6360000,'ms')),
 (207468, numpy.timedelta64(27480000,'ms')),
 (361428, numpy.timedelta64(12660000,'ms')),
 (398418, numpy.timedelta64(25260000,'ms')),
 (567918, numpy.timedelta64(21660000,'ms')),
 (659778, numpy.timedelta64(36060000,'ms')),
 (693392, numpy.timedelta64(3720000,'ms')),
 (791537, numpy.timedelta64(28860000,'ms')),
 (920657, numpy.timedelta64(8460000,'ms')),
 (920727, numpy.timedelta64(240000,'ms')),
 (937794, numpy.timedelta64(7260000,'ms')),
 (1047114, numpy.timedelta64(3660000,'ms')),
 (1062030, numpy.timedelta64(21300000,'ms')),
 (1081702, numpy.timedelta64(7740000,'ms')),
 (1156012, numpy.timedelta64(9060000,'ms')),
 (1248022, numpy.timedelta64(12660000,'ms')),
 (1471252, numpy.timedelta64(3660000,'ms')),
 (1501922, numpy.timedelta64(13860000,'ms')),
 (1506722, numpy.timedelta64(3660000,'ms'))]

In [25]:
# Check if everything adds up 

start_timestamp = np.datetime64(int(btcusd['timestamp'][0]), 'ms')

total_minutes = np.timedelta64(len(btcusd)-1,'m')

total_error = np.array([a[1] for a in missing_indices]).sum()

end_timestamp = np.datetime64(int(btcusd['timestamp'].values[-1]), 'ms')

In [26]:
# Time that still cant be accounted for : 22 minutes

end_timestamp - (start_timestamp + total_minutes + total_error)

numpy.timedelta64(-1320000,'ms')

## Resampling to relevant time horizon

In [79]:
def resample(df,interval):
    """
    Input : df -- dataframe OHLCV with pandas datetime as index.
            interval -- string eg. 1T,1H,1D,1m 
    
    Output : df resampled. 
    
    In the resample code, T -- minute eg 5T will resample for 5 min intervals,
    H -- hours, D -- days, m -- months.
    """
    
    ohlc_dict = {
        'open':'first',
        'high':'max',
        'low':'min',
        'close':'last',
        'volume':'sum'
        }
    
    df = df.resample(interval).agg(ohlc_dict)
    
    return(df)

## Some quick plotting

In [297]:
def plot_daily_price_volume(df):
    df_1d = resample(df,'1D')

    area_chart = px.area(df_1d.close, title = 'Daily price')
    
    area_chart.update_xaxes(title_text = 'Date')
    area_chart.update_yaxes(title_text = 'close price', tickprefix = '$')
    area_chart.update_layout(showlegend = False)
    area_chart.show()
    
    area_chart = px.area(df_1d.volume, title = 'Daily volume')

    area_chart.update_xaxes(title_text = 'Date')
    area_chart.update_yaxes(title_text = 'volume')
    area_chart.update_layout(showlegend = False)
    area_chart.show()

In [300]:
plot_daily_price_volume(btcusd)

In [18]:
#Rough maybe unnecessary code

## Naive change of time horizon. We can do something 
## more complex in the future.

#zdef get_new_time(df,th):
#    """ df : dataframe 
#        th : np.timedelta64 object"""
#    i = 0
#    m = len(df)
#    bad_indices = []
#    while i <= m-1:

#th = 5
#index_groups = []
#bad_indices = []
#
#i = 0
#m = len(date)
#
#temp = []
#
#while i <= m-1:
#    i_temp = i + th
#    if i_temp <= m-1 and date[i_temp] == date[i] + 60000*th:
#        temp.append(i)
#        i = i_temp
#    else:
#        index_groups.append(temp)
#        temp = []
#        bad_indices.append(i)
#        i += 1    

## Pumps 

Let us assume a pump is a change which is above the 90th percentile. We want to see how often such a singular event 
sustains into the future and when its effect dies down. 

In [237]:
### Choose a threshold you are interested in. 

pump_thresh = 98

In [238]:
a = np.percentile(returns,pump_thresh)
a

0.0022365623268529988

In [335]:
def check_for_pump_action(df,pump_thresh):
    """
    input - 
        df : dataframe with a column for returns data.
        pump_thresh : pump threshold
        
    output -
        singular_indices : indices where the returns data is above 
                       the threshold
        p : dict {k : v} where k is a singular index and v is the last 
            index after k where the returns are positive.
           
    """
    a = np.percentile(df['returns'],pump_thresh)
    
    m = len(df)
    p = {}
    i = 0
    singular_indices = []
    while (i <= m-1):
        if df['returns'][i] >= a:
            singular_indices.append(i)
            t = i
            j = i + 1
            while df['returns'][j] >= 0:
                   j = j+1
            p[i] = j-1        
            i = j
        else:
            i += 1 
    singular_date_indices = df.index[singular_indices]        
    return(p,singular_indices,singular_date_indices)        

Ad hoc def : positive persistent time - length of interval post a singular event for which the returns data is positive. 

(singular event is an instance where the returns data is above our pump
threshold.)

In [256]:
pump_dict,singular_indices,singular_date_indices = check_for_pump_action(btcusd,a)

In [241]:
def get_list_of_persistent_time(pump_dict):
    """
    Input - 
        pump_dict :
        
    Output - 
        list with length of persistent times.
    """
    t = [pump_dict[k]-k for k in pump_dict.keys()]
    
    return(t)

In [401]:
# plot the number of singular events in each month from jan 2018. 

def count_singular_events(pump_dict,singular_date_indices,agg_time = '1m',agg_method = 'sum'): 
    """
    input : pump_dict - dict with k:v pairs (singular_index : last index after k where the returns are positive)
            singular_date_indices - indices where we have singular events (very high returns) 
            agg_time - we might want to study how the pump events evolve through time. So 
                       we split the dataset into chunks each of time interval size agg time. 
                       default - 1m = 1 month.
    output : 
            df_temp - dataframe with information about number of singular events, 
                      and the probability of the singular event being good - i.e. we have positive 
                      returns for a period of greater than 1. 
    """
    
    
    l = [1]*len(singular_date_indices)
    ppt_list = get_list_of_persistent_time(pump_dict)
    good_singular_indices = (np.array(ppt_list) >= 1).astype('int')

    df = pd.DataFrame({'date': singular_date_indices,'singular_event': l,'good_singular_event':good_singular_indices})
    df = df.set_index('date')
    
    #df = df.drop(['date'])
    df_temp = df.resample(agg_time).agg({'singular_event':agg_method,'good_singular_event':agg_method})
    df_temp['good_singular_event_prob'] = (df_temp['good_singular_event'] + .000001)/(df_temp ['singular_event'] + .000001)
    #df_temp['date in {}'.format(agg_time)] = df_temp.index
    df_temp['date'] = df_temp.index
    
    return(df_temp)

In [244]:
singular_count_df = count_singular_events(pump_dict,singular_date_indices,'1m',agg_method = 'sum')

In [263]:
singular_count_df.head()

,singular_event,good_singular_event,good_singular_event_prob,date
date,,,,
2018-02-28,1664,734,0.441106,2018-02-28
2018-03-31,2237,933,0.417076,2018-03-31
2018-04-30,942,382,0.405520,2018-04-30
2018-05-31,615,246,0.400000,2018-05-31
2018-06-30,432,170,0.393519,2018-06-30


In [246]:
fig = px.scatter(singular_count_df,x = 'date',y = 'singular_event',
                 labels = {'date':'date in months','singular_event':'singular events count aggregated over months'},
                title = 'Number of singular events in a month')
fig.show()

Ignore the last data point - the one for february. 

In [247]:
fig = px.scatter(singular_count_df,x = 'date',y = 'good_singular_event_prob',
                 labels = {'date':'date in months','singular_event_prob':'good_singular_event_prob'},
                title = 'Probability of positive returns of length atleast 1 following a singular event')
fig.show()

Ignoring the data point for february, the last three months witness probabilities of positive 
returns in excess of 50%. 

In [221]:
#count_singular_events(singular_date_indices,'1m',agg_method = 'sum')

In [257]:
ppt_list = get_list_of_persistent_time(pump_dict)

In [258]:
def draw_ppt_hist(ppt_list,interval_measure):    
    fig = px.histogram(pd.DataFrame({'x' : ppt_list}),x = 'x',
                       nbins = 30,
                       labels = {'x' : interval_measure}
                      ,title = 'Length of positive returns interval following a singular event')
    fig.show()

In [301]:
draw_ppt_hist(ppt_list,'minutes')

NOTE : negative returns in the instance just after the singular event are counted under 0.
    
Over the course of the data, for the majority of times, we have a negative returns following a singular event. Perhaps if we could figure out how to identify those events which result 
in positive returns of greater than 5 - bit of a needle in the haystack. 

In [260]:
def scatter_ppt_list(ppt_list,singular_date_indices,agg_time,agg_method,title):
    """
    Input :
        ppt_list - list : list of positive persistent times over the course of the dataset.
        singular_date_indices - datetime list : the corresponding date indices
        agg_time : string eg '5T','1D','1m'
        agg_method : 'mean' or 'median' or 'max' or 'min'
        
    Output:
        
         
    """
    
    df = pd.DataFrame({'date': singular_date_indices,'positive persistent time':ppt_list})
    df = df.set_index('date')
    df_temp = df.resample(agg_time).agg({'positive persistent time':agg_method})
    df_temp['date in {}'.format(agg_time)] = df_temp.index
    fig = px.scatter(df_temp,x = 'date in {}'.format(agg_time), y="positive persistent time",labels = 
                     {'positive persistent time' : 'positive persistent time with agg method : {}'.format(agg_method)},title = title)
    fig.show()

In [262]:
## Plot the positive persistent time averaged over every month from jan 2018. 

scatter_ppt_list(ppt_list,singular_date_indices,'1m','mean','Aggregated length of interval with positive returns')

### Comment : 

It seems for 1 minute data, this positive persistence doesn't increase over time and in fact one can expect with more than 50% chance that the returns following a huge spike will be negative. 

- I thought this would gradually increase as we approached this years bull run. 

## Code to automate the analysis above. 

This section needs work as some preliminary tests show that it might not be entirely 
meaningful to study singular events in the manner above. 

In [377]:
def plot_number_of_singular_events_monthly(singular_count_df):
    fig = px.scatter(singular_count_df,x = 'date',y = 'singular_event',
                 labels = {'date':'date in months','singular_event':'singular events count aggregated over months'},
                title = 'Number of singular events in a month')
    fig.show()

In [378]:
def plot_prob_of_positive_returns(singular_count_df):
    fig = px.scatter(singular_count_df,x = 'date',y = 'good_singular_event_prob',
                 labels = {'date':'date in months','singular_event_prob':'good_singular_event_prob'},
                title = 'Probability of positive returns of length atleast 1 following a singular event')
    fig.show()

In [368]:
def load_resample_df(path,time_measure):
    df = pd.read_csv(path)
    df.columns = ['timestamp','open','high','low','close','volume']
    df['date'] = pd.to_datetime(df['timestamp'], unit = 'ms')
    df = df.set_index(pd.DatetimeIndex(df['date']))
    df = df.drop(['date'],axis = 1) 
     
    _,_ = check_for_missing_timestamps(df)
    df = resample(df,time_measure)
    
    df['returns'] = df['open']
    df['returns'] = df['open'].pct_change()
    print('')
    
    df = df.dropna(axis = 0)
    
    print(df.head())
    return(df)

In [395]:
df = load_resample_df(path,'30T')

Number of instances for which we have intervals of missing data is 22.

Total amount of missing time in dataframe is 5160.963833333333 minutes

                        open     high      low    close      volume   returns
date                                                                         
2018-02-09 10:30:00  8185.06  8320.00  8180.00  8264.99  613.026745 -0.005516
2018-02-09 11:00:00  8258.10  8339.00  8227.82  8290.87  615.647203  0.008924
2018-02-09 11:30:00  8290.90  8395.00  8250.00  8288.03  736.206805  0.003972
2018-02-09 12:00:00  8288.03  8300.00  8201.95  8236.44  513.958523 -0.000346
2018-02-09 12:30:00  8231.04  8286.06  8219.95  8241.00  489.291033 -0.006876


In [396]:
pump_dict,singular_indices,singular_date_indices = check_for_pump_action(df,pump_thresh)

In [397]:
singular_count_df = count_singular_events(pump_dict,singular_date_indices,'1m',agg_method = 'sum')

In [398]:
singular_count_df

,singular_event,good_singular_event,good_singular_event_prob,date
date,,,,
2018-02-28,60,36,6.000000e-01,2018-02-28
2018-03-31,75,36,4.800000e-01,2018-03-31
2018-04-30,34,21,6.176471e-01,2018-04-30
2018-05-31,17,7,4.117647e-01,2018-05-31
2018-06-30,18,9,5.000000e-01,2018-06-30
2018-07-31,8,4,5.000001e-01,2018-07-31
2018-08-31,17,10,5.882353e-01,2018-08-31
2018-09-30,8,1,1.250001e-01,2018-09-30
2018-10-31,5,1,2.000002e-01,2018-10-31


In [399]:
def study_persistence(path,time_measure,pump_thresh):
    df = load_resample_df(path,time_measure)
    
    returns = df['returns']
    plot_daily_price_volume(df)
    
    pump_dict,singular_indices,singular_date_indices = check_for_pump_action(df,pump_thresh)
    
    singular_count_df = count_singular_events(pump_dict,singular_date_indices,'1m',agg_method = 'sum')
    
    plot_number_of_singular_events_monthly(singular_count_df)
    
    plot_prob_of_positive_returns(singular_count_df)
    
    ppt_list = get_list_of_persistent_time(pump_dict)
    
    draw_ppt_hist(ppt_list,time_measure)
    
    scatter_ppt_list(ppt_list,singular_date_indices,'1m','mean','Aggregated length of interval with positive returns')
    

In [402]:
path = '../Binance/crypto_pumps/data/raw/Binance/BTC_USDT.csv'

df = study_persistence(path,'45T',98)

Number of instances for which we have intervals of missing data is 22.

Total amount of missing time in dataframe is 5160.963833333333 minutes

                        open     high      low    close       volume   returns
date                                                                          
2018-02-09 10:30:00  8185.06  8320.00  8180.00  8264.29   877.077481 -0.005516
2018-02-09 11:15:00  8272.83  8395.00  8250.00  8288.03  1087.803272  0.010723
2018-02-09 12:00:00  8288.03  8300.00  8201.95  8260.00   714.835493  0.001837
2018-02-09 12:45:00  8264.99  8314.99  8219.95  8279.85   867.720873 -0.002780
2018-02-09 13:30:00  8279.85  8422.50  8279.00  8395.00  1195.904688  0.001798
